In [1]:
from dotenv import load_dotenv
from IPython.display import display, HTML, JSON, Markdown
load_dotenv()

True

In [2]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///resources/Chinook.db")

In [3]:
from langchain.tools import tool

@tool
def sql_query(query: str) -> str:

    """Obtain information from the database using SQL queries"""

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

sql_query.invoke("SELECT * FROM Artist LIMIT 10")

"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

In [8]:
from langchain.agents import create_agent
from langchain_aws import ChatBedrock

# 1. CONFIGURACIÓN PARA DEEPSEEK-R1 (Razonamiento Complejo)
# Ideal para agentes que necesitan planificar pasos lógicos.
# llm = ChatBedrock(
#     model_id="us.deepseek.r1-v1:0",  # ID oficial validado
#     region_name="us-east-1",
#     model_kwargs={
#         "temperature": 0.6, # DeepSeek recomienda 0.6 para razonamiento
#         "max_tokens": 8192,  # Recomendado para no degradar calidad del CoT
#         "top_p": 0.95,
#     }
# )


# llm = ChatBedrock(
#     model_id="us.deepseek.v3-v1:0", # Prueba este primero
#     region_name="us-east-1",        # O us-west-2
#     model_kwargs={
#         "temperature": 0.7,
#         "max_tokens": 4096
#     }
# )
# from langchain_aws import ChatBedrock
# llm = ChatBedrock(
# model_id="us.meta.llama4-scout-17b-instruct-v1:0",  # Nota el prefijo "us."
# # model_id="cohere.command-r-plus-v1:0",
# region_name="us-east-1",
# model_kwargs={
# "temperature": 0.5,
# "max_tokens": 2048,
# "top_p": 0.9,
# }
# )


# llm = ChatBedrock(
#     model_id="us.meta.llama4-maverick-17b-instruct-v1:0",  # Nota el prefijo "us."
#     region_name="us-east-1",
#     model_kwargs={
#         "temperature": 0.5,
#         "max_tokens": 2048,
#         "top_p": 0.9,
#     }
# )

llm = ChatBedrock(
    model_id="amazon.nova-lite-v1:0",  # Nota el prefijo "us."
    region_name="us-east-1",
    model_kwargs={
        "temperature": 0.5,
        "max_tokens": 2048,
        "top_p": 0.9,
    }
)


# Create the RAG agent
agent = create_agent(
    model=llm,
    tools=[sql_query]
    
)

In [9]:
from langchain.messages import HumanMessage

question = HumanMessage(content="Who is the most popular artist beginning with 'S' in this database?")

response = agent.invoke(
    {"messages": [question]}
)

In [10]:
from pprint import pprint

pprint(response['messages'])

[HumanMessage(content="Who is the most popular artist beginning with 'S' in this database?", additional_kwargs={}, response_metadata={}, id='7d41a22d-b762-48e1-8883-6c073d8cb66a'),
 AIMessage(content=[{'type': 'text', 'text': "<thinking>To find the most popular artist beginning with 'S', I need to query the database for artists whose names start with 'S' and then find the one with the highest popularity. The exact query will depend on the structure of the database, but it will likely involve a SELECT statement with a WHERE clause to filter by the first letter and an ORDER BY clause to sort by popularity.</thinking>\n"}, {'type': 'tool_use', 'name': 'sql_query', 'input': {'query': "SELECT artist_name, popularity FROM artists WHERE artist_name LIKE 'S%' ORDER BY popularity DESC LIMIT 1"}, 'id': 'tooluse_v_JX_baCQv-0JPsYTg5MRA'}], additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': 'dd15c1e3-85e0-4e68-b1d3-d519a0132f73', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date'

In [11]:
print(response["messages"][-3].tool_calls[0]['args']['query'])

SELECT artist_name, popularity FROM artists WHERE artist_name LIKE 'S%' ORDER BY popularity DESC LIMIT 1


In [12]:
# Check the message count
print(f"Total messages: {len(response['messages'])}")

# See all messages and their tool_calls
for i, msg in enumerate(response['messages']):
    tool_calls = getattr(msg, 'tool_calls', None)
    print(f"[{i}] {type(msg).__name__}: tool_calls={tool_calls}")

Total messages: 4
[0] HumanMessage: tool_calls=None
[1] AIMessage: tool_calls=[{'name': 'sql_query', 'args': {'query': "SELECT artist_name, popularity FROM artists WHERE artist_name LIKE 'S%' ORDER BY popularity DESC LIMIT 1"}, 'id': 'tooluse_v_JX_baCQv-0JPsYTg5MRA', 'type': 'tool_call'}]
[2] ToolMessage: tool_calls=None
[3] AIMessage: tool_calls=[]
